# Local Small CNN Model Trainer

Disclaimer: Initial code generated by MinRes AI (Open Web UI) model:gpt-5

Input data sorted by folders

In [20]:
#Verify GPU availability
!nvidia-smi

Wed Oct  8 17:58:49 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.247.01             Driver Version: 535.247.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060        Off | 00000000:01:00.0 Off |                  N/A |
| 37%   34C    P8              14W / 170W |     80MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [21]:
# import extensions
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

## 1. Data pipeline

Some preprocessing followed by piping data into data sets for training and validation

### 1.1 Transformations
preprocesing input data (e.g. resizing, grascale, etc)


In [22]:

transform = transforms.Compose([
    transforms.Grayscale(),              # if your digits are greyscale
    transforms.Resize((28, 28)),         # standardise size
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)) # normalise to [-1, 1]
])

### 1.2 Download Data
I download my data from my G Drive

In [23]:
# everything here is untracked by .gitignore btw
import gdown

url = "https://drive.google.com/uc?id=1slfq-_VI3d6QLGwa1zP1oRgxc5YZSWXQ"
output = "training_data/task3data.zip"
gdown.download(url, output, quiet=False)

!unzip -qo training_data/task3data.zip -d training_data/task3data/

train_data = datasets.ImageFolder(root='training_data/task3data/train', transform=transform)
val_data   = datasets.ImageFolder(root='training_data/task3data/valid', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=32, shuffle=False)

Downloading...
From: https://drive.google.com/uc?id=1slfq-_VI3d6QLGwa1zP1oRgxc5YZSWXQ
To: /home/21502396/Searle_Jack_21502396/model_training/training_data/task3data.zip
100%|██████████| 4.52M/4.52M [00:00<00:00, 6.07MB/s]


## 2. simple CNN architecture
create them layers

In [25]:
'''class DigitCNN(nn.Module):
    def __init__(self, num_classes):
        super(DigitCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 14x14

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)   # 7x7
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x'''

from my_model_defs import DigitCNN


### 3. Set up model/optim/loss

In [26]:
num_classes = len(train_data.classes)  # should match number of digit folders
model = DigitCNN(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


## 4. training loop

In [27]:
for epoch in range(50):  # adjust epochs as needed
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")


Epoch 1, Loss: 1.8517
Epoch 2, Loss: 1.1802
Epoch 3, Loss: 0.9644
Epoch 4, Loss: 0.7548
Epoch 5, Loss: 0.6329
Epoch 6, Loss: 0.5413
Epoch 7, Loss: 0.4097
Epoch 8, Loss: 0.3486
Epoch 9, Loss: 0.2705
Epoch 10, Loss: 0.2414
Epoch 11, Loss: 0.2594
Epoch 12, Loss: 0.2058
Epoch 13, Loss: 0.1699
Epoch 14, Loss: 0.1658
Epoch 15, Loss: 0.1389
Epoch 16, Loss: 0.1378
Epoch 17, Loss: 0.1307
Epoch 18, Loss: 0.1078
Epoch 19, Loss: 0.1043
Epoch 20, Loss: 0.0832
Epoch 21, Loss: 0.0763
Epoch 22, Loss: 0.0788
Epoch 23, Loss: 0.0627
Epoch 24, Loss: 0.0749
Epoch 25, Loss: 0.0596
Epoch 26, Loss: 0.0648
Epoch 27, Loss: 0.0587
Epoch 28, Loss: 0.1142
Epoch 29, Loss: 0.0712
Epoch 30, Loss: 0.0561
Epoch 31, Loss: 0.0635
Epoch 32, Loss: 0.0673
Epoch 33, Loss: 0.0662
Epoch 34, Loss: 0.0449
Epoch 35, Loss: 0.0368
Epoch 36, Loss: 0.0448
Epoch 37, Loss: 0.0292
Epoch 38, Loss: 0.0305
Epoch 39, Loss: 0.0411
Epoch 40, Loss: 0.0166
Epoch 41, Loss: 0.0865
Epoch 42, Loss: 0.0804
Epoch 43, Loss: 0.0727
Epoch 44, Loss: 0.03

## 5. Validate Loop

In [28]:
# Validation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Validation Accuracy: {100 * correct / total:.2f}%")

Validation Accuracy: 88.18%


## 6. Save model

In [30]:
#subject to change as I continue to tweak model
torch.save(model, "digit_classifier.pth")
